## Pre-trained models and datasets for audio classification

## Keyword spotting (KWS)

### Minds-14 (Dataset)

In [13]:
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", name="en-US", split="train")

/home/fds/miniconda3/envs/py310_pytorch/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [14]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14"
)

In [15]:
classifier(minds[0]["audio"])

[{'score': 0.9984146356582642, 'label': 'joint_account'},
 {'score': 0.00043745661969296634, 'label': 'business_loan'},
 {'score': 0.0004359595768619329, 'label': 'cash_deposit'},
 {'score': 0.0001263972808374092, 'label': 'atm_limit'},
 {'score': 0.00010039484914159402, 'label': 'balance'}]

In [16]:
id2label_fn = minds.features["intent_class"].int2str

In [17]:
id2label_fn(minds[0]["intent_class"])

'joint_account'

### Speech Commands (Dataset)

In [18]:
from datasets import load_dataset

speech_commands = load_dataset(
    "speech_commands", "v0.02", split="validation", streaming=True
)
sample = next(iter(speech_commands))

In [19]:
classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2"
)
classifier(sample["audio"].copy())

config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/342M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

[{'score': 0.9999892711639404, 'label': 'backward'},
 {'score': 1.7504938796264469e-06, 'label': 'happy'},
 {'score': 6.703033363919531e-07, 'label': 'follow'},
 {'score': 5.805901537314639e-07, 'label': 'stop'},
 {'score': 5.614546694232558e-07, 'label': 'up'}]

In [24]:
speech_commands_id2label_fn = speech_commands.features["label"].int2str

In [26]:
speech_commands_id2label_fn(sample["label"])

'backward'

## Language identification (LID)

### FLEURS (Few-shot Learning Evaluation of Universal Representations of Speech) (Dataset)

## Zero-Shot Audio Classification

### Environmental Speech Challenge (ESC) (Dataset)